In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import Model_CNN
import matplotlib.pyplot as plt
import numpy as np
import cv2
import csv

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Dropout
from keras import backend as K
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras.models import load_model

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12918993027440423803
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3164969369
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13480577495071071783
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
class NetCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
 
        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        model.add(Dropout(0.3))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

In [ ]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

with open("..\\data\\data\\csv\\galaxy\\galaxy_label_data_set.csv", 'rt') as csvFile:
    reader = csv.reader(csvFile, delimiter=",")
    # loop over the input images
    for index, row in enumerate(reader):
        if index in range(1,1000):
            # load the image, pre-process it, and store it in the data list
            file = "..\\data\\data\\images\\"+row[0]+".jpg"
            image = cv2.imread(file)
            image = cv2.resize(image, (212, 212))
            image = img_to_array(image)
            data.append(image)
            label = (0 if row[1] == 'smooth' else 1)
            labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [3]:
EPOCHS = 20
INIT_LR = 1e-3
BS = 25

# initialize the model
print("[INFO] compiling model...")
model = NetCNN.build(width=212, height=212, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

tensorboard = keras.callbacks.TensorBoard(log_dir='logs/LR1e3', batch_size=batch,
                                          histogram_freq=0, write_graph=True, write_images=True)

# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS,  callbacks=[tensorboard], verbose=1)

# save the model to disk
print("[INFO] serializing network...")
model.save("model.hdf5")

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Smooth/Spiral")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot3_dropout.png")

[INFO] loading images...
[INFO] compiling model...
[INFO] training network...
Epoch 1/20
29/29 [==============================] - 17s 593ms/step - loss: 0.9991 - acc: 0.5214 - val_loss: 0.6925 - val_acc: 0.4600
Epoch 2/20
29/29 [==============================] - 8s 282ms/step - loss: 0.6556 - acc: 0.6461 - val_loss: 0.5693 - val_acc: 0.7920
Epoch 3/20
29/29 [==============================] - 8s 269ms/step - loss: 0.6075 - acc: 0.7335 - val_loss: 0.5179 - val_acc: 0.8320
Epoch 4/20
29/29 [==============================] - 8s 270ms/step - loss: 0.5686 - acc: 0.7583 - val_loss: 0.4964 - val_acc: 0.7560
Epoch 5/20
29/29 [==============================] - 8s 267ms/step - loss: 0.5227 - acc: 0.7705 - val_loss: 0.4221 - val_acc: 0.8560
Epoch 6/20
29/29 [==============================] - 8s 266ms/step - loss: 0.4519 - acc: 0.8053 - val_loss: 0.4546 - val_acc: 0.8360
Epoch 7/20
29/29 [==============================] - 8s 267ms/step - loss: 0.4505 - acc: 0.8040 - val_loss: 0.3828 - val_acc: 0.86